<h3>Import Libraries for the Analysis</h3>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.backends.backend_pdf import PdfPages

import datetime

# import k-means clustering
from sklearn.cluster import KMeans
print("Libraries Imported")

Libraries Imported


<h3>Import All CSV Files</h3>

In [9]:
df_FoodFacilities = pd.read_csv('Food_Inspection_Facilities.csv')
df_FoodFacilities.head()

,X,Y,OBJECTID,FacilityMasterID,FacilityName,Category,CategoryStyle,SiteStreet,SiteCity,SiteTelephone,WorkArea
0,-80.358081,43.395509,11468,E4665DDB-2876-4C32-8C80-500F1776451B,KING VARIETY,"Food, General",Food store convenience / Variety,720 KING ST E,Cambridge,(519) 653-5943,CK2
1,-80.555346,43.531847,11469,9625C2A9-A439-4C27-AE95-669C4FA4C8A9,TIM HORTONS,"Food, General",Restaurant,1229 KING ST N,St. Jacobs,(519) 664-1544,KW8
2,-80.525369,43.484231,11471,126A3305-190C-4D61-B0E8-889AB4E101EA,Lakelands Diet Delivery,"Food, General",Caterer / Commissary,401 Weber ST N,Waterloo,(226) 444-5163,KW10
3,-80.536425,43.414992,11473,2E4D9684-C487-4F93-8AD5-1C5364A8FDAD,DRIFTWOOD PARK CHILD CARE CENTRE,"Food, Institutional",Child Care Onsite Prep./Recreational Camp/Co-op,50 PARKLAND CRES,Kitchener,(519) 570-2520,KW4
4,-80.444505,43.452131,11477,E65B7458-F178-4285-9926-F46520CAD2B0,PIZZA PIZZA,"Food, General",Restaurant,1020 OTTAWA ST N,Kitchener,(519) 748-2755,KW3


In [10]:
df_Infractions = pd.read_csv('Infractions_OpenData.csv', engine='python') 
#this file has an encoding issue and required specifying the engine to parse the data
df_Infractions.head()

,INFRACTION_ID,INSPECTION_ID,INFRACTION_TYPE,Infraction,Result,Comment,InspectionDate
0,5BCFD0AC-C034-4DC9-B8B4-02AF76EF5274,B62FEEB3-37A1-425F-ACEA-1EED58DFDB3C,NON-CRITICAL,"Furniture, equipment and appliances in food ha...",Corrected During Inspection,Maintain and arrange appliances to permit a cl...,2018/08/28
1,745D9715-50D2-4527-933A-0845C005500F,CAD3CDA5-505B-4718-A9AD-1BDCD4B0F532,NON-CRITICAL,"Furniture, equipment and appliances in food ha...",Not in Compliance,Remove all materials not associated with the d...,2018/08/23
2,A7A74F2D-1D62-4956-B764-0E998F8B6B50,A82C82A7-E485-49A2-A9F3-E552C51D9A97,NON-CRITICAL,"Furniture, equipment and appliances in food ha...",Not in Compliance,NaN,2018/09/20
3,0187B88A-1D44-430B-8578-15C1CB607391,BADE6395-E24F-4476-B4E4-F4EED578C1CF,NON-CRITICAL,"Furniture, equipment and appliances in food ha...",Not in Compliance,Maintain and arrange furniture to permit a cle...,2018/08/28
4,38AF0445-45C2-407F-8E9D-2AEED6CDFA18,9A3E9605-453A-46D2-8F75-B9AC22480A3B,NON-CRITICAL,"Furniture, equipment and appliances in food ha...",Not in Compliance,Maintain and arrange equipment to permit a cle...,2018/09/13


In [13]:
df_Inspections = pd.read_csv('Inspections_OpenData.csv')
df_Inspections.head()

,INSPECTION_ID,FACILITYID,INSPECTION_DATE,REQUIRE_REINSPECTION,CERTIFIED_FOOD_HANDLER,INSPECTION_TYPE
0,86BF1EB9-F088-484C-9725-00014ED539CA,98312352-BEF2-4695-8B7A-2DDAEC387210,2019/07/24,N,No,Compliance Inspection
1,78D6795D-A908-429B-95A4-00057AD61B4E,4AF8E99A-22AD-4FC6-9568-B161A06499CF,2019/08/21,N,No,Re-inspection
2,CB8A02EC-4EE2-4B64-90C2-00075CCC7428,D4FF8AF3-0D41-4F36-9B5E-48E599884E30,2018/11/09,N,No,Compliance Inspection
3,90CB570C-4898-4F0E-977E-001BF93575AC,3A7076E6-7C2C-427D-A107-7EFCB10E2DFE,2019/01/16,N,No,Re-inspection
4,AE6CB55D-FE36-4AD5-91A1-002B902E6FF0,F0C0D913-3724-4ACA-89D2-9E39FF44CF17,2019/09/25,N,No,Compliance Inspection


<h3>Set up Foursquare Endpoints</h3>

In [38]:
# @hidden_cell
CLIENT_ID = 'REMOVED FOR UPLOAD' # your Foursquare ID
CLIENT_SECRET = 'REMOVED FOR UPLOAD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

<h4>Function to Import from Venues Endpoint (free access)</h4>

In [64]:
#set up dataframe for Venues DataFrame
column_headings = ['FacilityMasterID', 'FoursquareID', 'FoursquareName']

# instantiate the dataframe
foursquare_venues = pd.DataFrame(columns=column_headings)
foursquare_venues.head()

,FacilityMasterID,FoursquareID,FoursquareName


In [74]:
def getVenues(name, latitudes, longitudes):
          
    INTENT = 'match' #find near exact match to the information provided
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&intent={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitudes, 
        longitudes,
        name,
        INTENT)

    # make the GET request
    results = requests.get(url).json()["response"]["venues"][0]
    FoursquareID = results['id']
    FoursquareName= results['name']
    
    return(FoursquareID, FoursquareName)

<h4>Test the function</h4>

In [75]:
getVenues(df_FoodFacilities.FacilityName[0],
                   df_FoodFacilities.Y[0], df_FoodFacilities.X[0])

('4d1b0a4eeab1224b32d6cee2', 'King Variety')

In [76]:
#Note that we will only populate this data after preliminary analysis to ensure that usage limits are not exceeded

0